In [16]:
import pandas as pd 
import numpy as np

In [17]:
data = pd.read_excel("search_comments_2025-10-19.xlsx")
data.head(1)

,comment_id,create_time,ip_location,note_id,content,user_id,nickname,avatar,sub_comment_count,pictures,parent_comment_id,last_modify_ts,like_count
0,68eb404500000000380293b5,1760247878000,新疆,68ea0b460000000003010908,1、你持续数年的自我对话本身也是一种行动。ai错了。2、质疑是科学精神的内核，质疑本身没错。...,62c6d4b3000000000303df86,伊布司机,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,444,NaN,0,1760879462285,8460


In [18]:
#数据的内容
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   comment_id         910 non-null    object
 1   create_time        910 non-null    int64 
 2   ip_location        78 non-null     object
 3   note_id            910 non-null    object
 4   content            904 non-null    object
 5   user_id            910 non-null    object
 6   nickname           907 non-null    object
 7   avatar             910 non-null    object
 8   sub_comment_count  910 non-null    int64 
 9   pictures           59 non-null     object
 10  parent_comment_id  910 non-null    int64 
 11  last_modify_ts     910 non-null    int64 
 12  like_count         910 non-null    int64 
dtypes: int64(5), object(8)
memory usage: 92.6+ KB


In [19]:
# 查找缺失值
missing_value = data.isnull().sum()/len(data)*100
# ip_location 和 pictures缺失值多，删除这两列
# 由于已经存在user_id, nickname可以考虑删除·
missing_value

comment_id            0.000000
create_time           0.000000
ip_location          91.428571
note_id               0.000000
content               0.659341
user_id               0.000000
nickname              0.329670
avatar                0.000000
sub_comment_count     0.000000
pictures             93.516484
parent_comment_id     0.000000
last_modify_ts        0.000000
like_count            0.000000
dtype: float64

In [20]:
#删除相关性低且缺失值大的列
data.columns = data.columns.str.strip()
needs_columns = ["comment_id", "create_time", "note_id", "content", "user_id", "sub_comment_count", "like_count"]
data = data[needs_columns]

In [21]:
#删除有空值的行
data = data.dropna().reset_index(drop=True)
print(f'删除空值后剩余行数: {len(data)}')

删除空值后剩余行数: 904


In [ ]:
# 检查用户与content是否有重复值
col = ['user_id','content']

# 重复值数量（被标记为重复的行数）
dup_rows = data[col].duplicated().sum()
print(f"列 {col} 标记为重复的行数: {dup_rows}")

# 查看哪些具体的值出现了多次（以及出现次数）
vc = data[col].value_counts()
print('出现次数大于1的值及其次数：')
print(vc[vc > 1])


列 ['user_id', 'content'] 标记为重复的行数: 40
出现次数大于1的值及其次数：
user_id                   content                                                                                              
5678e7ea4476082aec5da222  刚去试了一下 我滴乖乖。这么牛的嘛。已经安利给群里姐妹们了？[偷笑R][偷笑R][偷笑R]                                                            2
5dd0d673000000000100ab08  @池早 这个好  像我之前往自己的聊天框的记录  因为微信卸载啥都没了呜呜                                                                    2
5ae1972ee8ac2b52a71bbc4c  请问能不能导入以前在别的的软件写日记🥺                                                                                      2
5ae3e84c4eacab6c07dbbe08  就等这个了，帮我再进一步                                                                                             2
5481caefd6e4a97d79e046cb  需要会员吗？                                                                                                   2
5d8ae7e8000000000101a198  想问一下有没有云保存啊                                                                                              2
5e92fdda0000000001008

,user_id,content,count
0,5678e7ea4476082aec5da222,刚去试了一下 我滴乖乖。这么牛的嘛。已经安利给群里姐妹们了？[偷笑R][偷笑R][偷笑R],2
1,5dd0d673000000000100ab08,@池早 这个好 像我之前往自己的聊天框的记录 因为微信卸载啥都没了呜呜,2
2,5ae1972ee8ac2b52a71bbc4c,请问能不能导入以前在别的的软件写日记🥺,2
3,5ae3e84c4eacab6c07dbbe08,就等这个了，帮我再进一步,2
4,5481caefd6e4a97d79e046cb,需要会员吗？,2


In [11]:
# 删除 content 列中重复的评论，只保留第一次出现的
data = data.drop_duplicates(subset=['content'], keep='first').reset_index(drop=True)
print(f'去重后剩余行数: {len(data)}')
data.head()

去重后剩余行数: 864


,comment_id,create_time,note_id,content,user_id,sub_comment_count,like_count
0,68eb404500000000380293b5,1760247878000,68ea0b460000000003010908,1、你持续数年的自我对话本身也是一种行动。ai错了。2、质疑是科学精神的内核，质疑本身没错。...,62c6d4b3000000000303df86,444,8460
1,68ef004d0000000039007c56,1760493646000,68ea0b460000000003010908,评论区让我放心了，人类不会被AI取代。[笑哭R]当人们出现迷茫或者对事物有不清楚的认知时，会...,6196655a0000000010008eac,12,590
2,68ef702700000000370128a9,1760522281000,68ea0b460000000003010908,别太把ai的评价当回事儿，ai说我特别有哲学思维，很有洞见，总之就是很牛逼。给你看一下我导师...,5927beea50c4b47e9573156a,16,478
3,68f014080000000039024c38,1760564232000,68ea0b460000000003010908,很同意此刻的最高赞，列出了六条有逻辑、循序渐进的comment。很高兴看到作者也很赞成。但是...,6107f95e000000000100b2d6,3,151
4,68efd60700000000380125bb,1760548360000,68ea0b460000000003010908,人们什么时候才能明白，ai只是你给出的信息的镜子，而不是你这个人完整的本身。,5c62f55f000000001202c238,2,146


In [ ]:
#检查是否有多个用户留言
col = ['user_id']

# 重复值数量（被标记为重复的行数）
dup_rows = data[col].duplicated().sum()
print(f"列 {col} 标记为重复的行数: {dup_rows}")

# 查看哪些具体的值出现了多次（以及出现次数）
vc = data[col].value_counts()
print('出现次数大于1的值及其次数：')

df_duplicates = vc[vc > 1].reset_index()
df_duplicates.columns = col + ['count'] 
df_duplicates


#可以看到有一位用户有12个相关的评论，重复客户评论的次数多
#重复评论用户: 1. 乐于在社交媒体进行评论 2. 笔记系统的爱好者 =》可以与此类用户进行定性分析，访谈能够有更好的收获

列 ['user_id'] 标记为重复的行数: 95
出现次数大于1的值及其次数：
user_id                 
60e3ff1700000000200261e8    12
634bbfec000000001802b71b     4
603b0e2700000000010015cd     3
5c5ed45800000000100005f1     3
5678e7ea4476082aec5da222     3
                            ..
6652ea300000000007006a83     2
6655bab00000000003031db3     2
665ee3f70000000007005847     2
66695eee000000000700443e     2
6677d14300000000030315c5     2
Name: count, Length: 72, dtype: int64


,user_id,count
0,60e3ff1700000000200261e8,12
1,634bbfec000000001802b71b,4
2,603b0e2700000000010015cd,3
3,5c5ed45800000000100005f1,3
4,5678e7ea4476082aec5da222,3
...,...,...
67,6652ea300000000007006a83,2
68,6655bab00000000003031db3,2
69,665ee3f70000000007005847,2
70,66695eee000000000700443e,2


In [29]:
df_duplicates.to_excel('duplicate.xlsx',index=False)

In [31]:
# 查看 user_id 为 '60e3ff1700000000200261e8' 的所有评论内容
target_id = '60e3ff1700000000200261e8'
user_comments = data[data['user_id'] == target_id][['user_id', 'content']]
print(user_comments)

#本以为有什么特别的发现，营销号之类的，原来是官号，好的可以删掉了lol，给忘记了

                      user_id  \
61   60e3ff1700000000200261e8   
79   60e3ff1700000000200261e8   
215  60e3ff1700000000200261e8   
335  60e3ff1700000000200261e8   
386  60e3ff1700000000200261e8   
395  60e3ff1700000000200261e8   
426  60e3ff1700000000200261e8   
428  60e3ff1700000000200261e8   
524  60e3ff1700000000200261e8   
581  60e3ff1700000000200261e8   
795  60e3ff1700000000200261e8   
836  60e3ff1700000000200261e8   

                                               content  
61                                              感谢推荐嗷！  
79                   宝感谢推荐呀，能不能加一下俺，给你送会员：shayujun1994  
215                 赞[赞R]记录的越多 越能洞察到适合自己的 Tag 结构[萌萌哒R]  
335                                         谢谢推荐鸭[飞吻R]  
386                                     太好了，感谢推荐鸭[飞吻R]  
395    哇哇哇，写得很棒！夺回情绪主动权这个概念很好～～另外我们最近的👸号更新里也有写怎么记录情绪日记  
426                                    也太厉害了吧！之前居然看漏了！  
428                 这么棒的分享，必须送个礼物感谢下！！！（可以单独戳戳我[暗中观察R]  
524  感谢宝推荐！[飞吻R][飞吻R][飞吻R]记录真实的自己越多，Ai洞察就越窝心呀，让记录的意...  


In [32]:
# 删除user_id为官号的评论
data = data[data["user_id"] != "60e3ff1700000000200261e8"].reset_index(drop=True)

In [33]:
data.shape

(892, 7)

In [ ]:
data.head()
data.to_excel("output.xlsx", index=False)